In [ ]:
#Make sure yo install requirements before running this
#requirements.txt contains the sparkmeasure package

In [2]:
#spark.stop()

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
spark = SparkSession\
    .builder\
    .appName("PythonSQL")\
    .config("spark.hadoop.fs.s3a.s3guard.ddb.region","us-east-2")\
    .config("spark.yarn.access.hadoopFileSystems","s3a://go01-demo")\
    .config("spark.jars.packages","ch.cern.sparkmeasure:spark-measure_2.12:0.23")\
    .config("spark.dynamicAllocation.enabled","false")\
    .getOrCreate()

:: loading settings :: url = jar:file:/runtime-addons/spark320-18-hf3-c1r4x9d/opt/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/cdsw/.ivy2/cache
The jars for the packages stored in: /home/cdsw/.ivy2/jars
ch.cern.sparkmeasure#spark-measure_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-1a357843-f956-4921-8f1e-28dcbaff0bd6;1.0
	confs: [default]
	found ch.cern.sparkmeasure#spark-measure_2.12;0.23 in central
	found com.fasterxml.jackson.module#jackson-module-scala_2.12;2.13.3 in central
	found com.fasterxml.jackson.core#jackson-core;2.13.3 in central
	found com.fasterxml.jackson.core#jackson-annotations;2.13.3 in central
	found com.fasterxml.jackson.core#jackson-databind;2.13.3 in central
	found com.thoughtworks.paranamer#paranamer;2.8 in central
	found org.slf4j#slf4j-api;1.7.36 in central
	found org.influxdb#influxdb-java;2.14 in central
	found com.squareup.retrofit2#retrofit;2.4.0 in central
	found com.squareup.retrofit2#converter-moshi;2.4.0 in central
	found com.squareup.moshi#moshi;1.5.0 in central
	found com.squareup.okio#okio;1.13

In [3]:
spark.sparkContext.getConf().getAll()

[('spark.dynamicAllocation.enabled', 'false'),
 ('spark.eventLog.enabled', 'true'),
 ('spark.hadoop.fs.s3a.s3guard.ddb.region', 'us-east-2'),
 ('spark.network.crypto.enabled', 'true'),
 ('spark.app.id', 'spark-application-1683253559571'),
 ('spark.app.initial.file.urls',
  'spark://100.100.249.154:39123/files/com.thoughtworks.paranamer_paranamer-2.8.jar,spark://100.100.249.154:39123/files/ch.cern.sparkmeasure_spark-measure_2.12-0.23.jar,spark://100.100.249.154:39123/files/org.apache.kafka_kafka-clients-3.2.1.jar,spark://100.100.249.154:39123/files/com.squareup.moshi_moshi-1.5.0.jar,spark://100.100.249.154:39123/files/com.squareup.retrofit2_retrofit-2.4.0.jar,spark://100.100.249.154:39123/files/org.lz4_lz4-java-1.8.0.jar,spark://100.100.249.154:39123/files/com.squareup.okhttp3_okhttp-3.11.0.jar,spark://100.100.249.154:39123/files/com.fasterxml.jackson.module_jackson-module-scala_2.12-2.13.3.jar,spark://100.100.249.154:39123/files/com.github.luben_zstd-jni-1.5.2-1.jar,spark://100.100.249

In [4]:
import os
print("https://spark-"+os.environ["CDSW_ENGINE_ID"]+"."+os.environ["CDSW_DOMAIN"])

https://spark-n7x9abcll1i6tdz3.ml-4c5feac0-3ec.go01-dem.ylcu-atmi.cloudera.site


In [5]:
from pyspark.sql.types import Row, StructField, StructType, StringType, IntegerType
# A list of Rows. Infer schema from the first row, create a DataFrame and print the schema
rows = [Row(name="John", age=19), Row(name="Smith", age=23), Row(name="Sarah", age=18)]
some_df = spark.createDataFrame(rows)
some_df.printSchema()

# A list of tuples
tuples = [("John", 19), ("Smith", 23), ("Sarah", 18)]

# Schema with two fields - person_name and person_age
schema = StructType([StructField("person_name", StringType(), False),
                    StructField("person_age", IntegerType(), False)])

# Create a DataFrame by applying the schema to the RDD and print the schema
another_df = spark.createDataFrame(tuples, schema)
another_df.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)

root
 |-- person_name: string (nullable = false)
 |-- person_age: integer (nullable = false)



In [6]:
import os
print("https://spark-"+os.environ["CDSW_ENGINE_ID"]+"."+os.environ["CDSW_DOMAIN"])

https://spark-n7x9abcll1i6tdz3.ml-4c5feac0-3ec.go01-dem.ylcu-atmi.cloudera.site


In [7]:
#spark.conf.get('spark.executor.cores')

In [8]:
another_df.write.mode("overwrite").partitionBy('person_age').saveAsTable("cmltable", format="parquet")

Hive Session ID = 300bf600-8e44-4c87-80f3-b47fb3bb0ceb


In [9]:
# Initialize sparkMeasure
# Load the Python API for sparkmeasure package
# and attach the sparkMeasure Listener for stagemetrics to the active Spark session

from sparkmeasure import StageMetrics
stagemetrics = StageMetrics(spark)

In [10]:
# The easiest way to start using sparkMesure is with the "runandmeasure" method
# This will execute your Spark action, return the results, and collect and aggregate execution metrics

stagemetrics.runandmeasure(globals(), """
spark.sql("select count(*) from cmltable").show()
""")

+--------+
|count(1)|
+--------+
|       3|
+--------+


Scheduling mode = FIFO
Spark Context default degree of parallelism = 2

Aggregated Spark stage metrics:
numStages => 2
numTasks => 3
elapsedTime => 904 (0.9 s)
stageDuration => 836 (0.8 s)
executorRunTime => 1013 (1 s)
executorCpuTime => 549 (0.5 s)
executorDeserializeTime => 257 (0.3 s)
executorDeserializeCpuTime => 233 (0.2 s)
resultSerializationTime => 0 (0 ms)
jvmGCTime => 28 (28 ms)
shuffleFetchWaitTime => 0 (0 ms)
shuffleWriteTime => 15 (15 ms)
resultSize => 4286 (4.2 KB)
diskBytesSpilled => 0 (0 Bytes)
memoryBytesSpilled => 0 (0 Bytes)
peakExecutionMemory => 0
recordsRead => 3
bytesRead => 2584 (2.5 KB)
recordsWritten => 0
bytesWritten => 0 (0 Bytes)
shuffleRecordsRead => 2
shuffleTotalBlocksFetched => 2
shuffleLocalBlocksFetched => 1
shuffleRemoteBlocksFetched => 1
shuffleTotalBytesRead => 150 (150 Bytes)
shuffleLocalBytesRead => 75 (75 Bytes)
shuffleRemoteBytesRead => 75 (75 Bytes)
shuffleRemoteBytesReadToDisk => 0 (0 By

In [11]:
# An equivalent API for collecting execution metrics is to explicitly wrap your Spark workload
# into stagemetrics instrumentation, as in this example

stagemetrics.begin()

df = spark.sql("select * from cmltable")
stagemetrics.end()

# Print a summary report
stagemetrics.print_report()


Scheduling mode = FIFO
Spark Context default degree of parallelism = 2

Aggregated Spark stage metrics:
numStages => 0
numTasks => 0
elapsedTime => -9223372036854775807 (-9223372036854775807 ms)
stageDuration => 0 (0 ms)
executorRunTime => 0 (0 ms)
executorCpuTime => 0 (0 ms)
executorDeserializeTime => 0 (0 ms)
executorDeserializeCpuTime => 0 (0 ms)
resultSerializationTime => 0 (0 ms)
jvmGCTime => 0 (0 ms)
shuffleFetchWaitTime => 0 (0 ms)
shuffleWriteTime => 0 (0 ms)
resultSize => 0 (0 Bytes)
diskBytesSpilled => 0 (0 Bytes)
memoryBytesSpilled => 0 (0 Bytes)
peakExecutionMemory => 0
recordsRead => 0
bytesRead => 0 (0 Bytes)
recordsWritten => 0
bytesWritten => 0 (0 Bytes)
shuffleRecordsRead => 0
shuffleTotalBlocksFetched => 0
shuffleLocalBlocksFetched => 0
shuffleRemoteBlocksFetched => 0
shuffleTotalBytesRead => 0 (0 Bytes)
shuffleLocalBytesRead => 0 (0 Bytes)
shuffleRemoteBytesRead => 0 (0 Bytes)
shuffleRemoteBytesReadToDisk => 0 (0 Bytes)
shuffleBytesWritten => 0 (0 Bytes)
shuffleReco

In [12]:
stagemetrics.begin()

#new_df = df.coalesce(1)
df.collect()

stagemetrics.end()

# Print a summary report
stagemetrics.print_report()


Scheduling mode = FIFO
Spark Context default degree of parallelism = 2

Aggregated Spark stage metrics:
numStages => 1
numTasks => 2
elapsedTime => 380 (0.4 s)
stageDuration => 380 (0.4 s)
executorRunTime => 553 (0.6 s)
executorCpuTime => 159 (0.2 s)
executorDeserializeTime => 102 (0.1 s)
executorDeserializeCpuTime => 92 (92 ms)
resultSerializationTime => 0 (0 ms)
jvmGCTime => 0 (0 ms)
shuffleFetchWaitTime => 0 (0 ms)
shuffleWriteTime => 0 (0 ms)
resultSize => 3434 (3.4 KB)
diskBytesSpilled => 0 (0 Bytes)
memoryBytesSpilled => 0 (0 Bytes)
peakExecutionMemory => 0
recordsRead => 3
bytesRead => 4038 (3.9 KB)
recordsWritten => 0
bytesWritten => 0 (0 Bytes)
shuffleRecordsRead => 0
shuffleTotalBlocksFetched => 0
shuffleLocalBlocksFetched => 0
shuffleRemoteBlocksFetched => 0
shuffleTotalBytesRead => 0 (0 Bytes)
shuffleLocalBytesRead => 0 (0 Bytes)
shuffleRemoteBytesRead => 0 (0 Bytes)
shuffleRemoteBytesReadToDisk => 0 (0 Bytes)
shuffleBytesWritten => 0 (0 Bytes)
shuffleRecordsWritten => 0



In [13]:
stagemetrics.begin()

df = df.coalesce(1)
df.collect()

stagemetrics.end()

# Print a summary report
stagemetrics.print_report()


Scheduling mode = FIFO
Spark Context default degree of parallelism = 2

Aggregated Spark stage metrics:
numStages => 1
numTasks => 1
elapsedTime => 424 (0.4 s)
stageDuration => 424 (0.4 s)
executorRunTime => 389 (0.4 s)
executorCpuTime => 76 (76 ms)
executorDeserializeTime => 19 (19 ms)
executorDeserializeCpuTime => 13 (13 ms)
resultSerializationTime => 0 (0 ms)
jvmGCTime => 0 (0 ms)
shuffleFetchWaitTime => 0 (0 ms)
shuffleWriteTime => 0 (0 ms)
resultSize => 1740 (1740 Bytes)
diskBytesSpilled => 0 (0 Bytes)
memoryBytesSpilled => 0 (0 Bytes)
peakExecutionMemory => 0
recordsRead => 3
bytesRead => 4038 (3.9 KB)
recordsWritten => 0
bytesWritten => 0 (0 Bytes)
shuffleRecordsRead => 0
shuffleTotalBlocksFetched => 0
shuffleLocalBlocksFetched => 0
shuffleRemoteBlocksFetched => 0
shuffleTotalBytesRead => 0 (0 Bytes)
shuffleLocalBytesRead => 0 (0 Bytes)
shuffleRemoteBytesRead => 0 (0 Bytes)
shuffleRemoteBytesReadToDisk => 0 (0 Bytes)
shuffleBytesWritten => 0 (0 Bytes)
shuffleRecordsWritten => 0

In [19]:
print(spark.conf.get('spark.sql.shuffle.partitions'))

200


In [31]:
import os
print("https://spark-"+os.environ["CDSW_ENGINE_ID"]+"."+os.environ["CDSW_DOMAIN"])

https://spark-rc6ogaxo81567r3w.ml-4c5feac0-3ec.go01-dem.ylcu-atmi.cloudera.site


In [10]:
# This code is for Jupyter notebooks
# Define cell and line magic to wrap sparkmeasure instrumentation
# See example in the next cell

from IPython.core.magic import (register_line_magic, register_cell_magic, register_line_cell_magic)

@register_line_cell_magic
def sparkmeasure(line, cell=None):
    "run and measure spark workload. Use: %sparkmeasure or %%sparkmeasure"
    val = cell if cell is not None else line
    stagemetrics.begin()
    eval(val)
    stagemetrics.end()
    stagemetrics.print_report()

In [11]:
from sparkmeasure import TaskMetrics
taskmetrics = TaskMetrics(spark)

taskmetrics.begin()
spark.sql("select count(*) from range(1000) cross join range(1000) cross join range(1000)").show()
taskmetrics.end()

taskmetrics.print_report()

+----------+
|  count(1)|
+----------+
|1000000000|
+----------+


Scheduling mode = FIFO
Spark Context default degree of parallelism = 2

Aggregated Spark task metrics:
numTasks => 5
successful tasks => 5
speculative tasks => 0
taskDuration => 872 (0.9 s)
schedulerDelayTime => 35 (35 ms)
executorRunTime => 778 (0.8 s)
executorCpuTime => 766 (0.8 s)
executorDeserializeTime => 59 (59 ms)
executorDeserializeCpuTime => 31 (31 ms)
resultSerializationTime => 0 (0 ms)
jvmGCTime => 0 (0 ms)
shuffleFetchWaitTime => 0 (0 ms)
shuffleWriteTime => 0 (0 ms)
gettingResultTime => 0 (0 ms)
resultSize => 2659 (2.6 KB)
diskBytesSpilled => 0 (0 Bytes)
memoryBytesSpilled => 0 (0 Bytes)
peakExecutionMemory => 0
recordsRead => 2000
bytesRead => 0 (0 Bytes)
recordsWritten => 0
bytesWritten => 0 (0 Bytes)
shuffleRecordsRead => 2
shuffleTotalBlocksFetched => 2
shuffleLocalBlocksFetched => 1
shuffleRemoteBlocksFetched => 1
shuffleTotalBytesRead => 148 (148 Bytes)
shuffleLocalBytesRead => 74 (74 Bytes)
shuffleRe